In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier

import helper
import text_embeddings
import performance
import models
import predict

# 1. Load data

In [2]:
x, y = helper.load_data()

# 2. Split data into train/test

In [3]:
x_train, x_test, y_train, y_test = helper.split_data(x, y)

# 3. Text embedding

In [4]:
x_train_features, x_test_features, vectorizer = text_embeddings.encode_tdfif(x_train, x_test, 'text')

# 4. Grid Search using 

-  https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier
-  https://scikit-learn.org/stable/modules/kernel_approximation.html  

In [5]:
final_x_train = x_train_features['text']['data']
final_y_train = y_train

final_x_test = x_test_features['text']['data']
final_y_test = y_test

In [6]:
model_params = dict(eta0=0.0001,loss='hinge', random_state=15, penalty='l2', tol=1e-3, verbose=0)
model_class = SGDClassifier

In [7]:
%%time
import matplotlib.pyplot as plt
# from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import GridSearchCV
from scipy.stats import randint as sp_randint


alpha_range = [ 0.0001,0.001,0.01,0.1,1,2,3,4,5,6,7,8,9,10,50,100]

model = model_class(**model_params)

search_params = {'alpha' : alpha_range}
search = GridSearchCV(model,
                      search_params,
                      cv=5,
                      scoring="f1",
                     return_train_score=True)


search.fit(final_x_train, final_y_train)
results = pd.DataFrame.from_dict(search.cv_results_)
results = results.sort_values(['param_alpha'])
results['mean_train_score-mean_test_score'] = results['mean_train_score'] - results['mean_test_score']
results[['param_alpha','mean_train_score','mean_test_score', 'mean_train_score-mean_test_score']]

/Users/aneeshmakala/Documents/ComputerScience/datascience/venv_datascience/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


CPU times: user 20 s, sys: 1.62 s, total: 21.6 s
Wall time: 13.1 s


param_alpha  mean_train_score  mean_test_score  \
0       0.0001          0.928729         0.920119   
1        0.001          0.874013         0.871768   
2         0.01          0.016937         0.016638   
3          0.1          0.000548         0.000325   
4            1          0.216872         0.215319   
5            2          0.263781         0.263230   
6            3          0.238686         0.238578   
7            4          0.237883         0.237877   
8            5          0.237882         0.237877   
9            6          0.237882         0.237877   
10           7          0.237882         0.237877   
11           8          0.237882         0.237877   
12           9          0.237882         0.237877   
13          10          0.237882         0.237877   
14          50          0.237882         0.237877   
15         100          0.237882         0.237877   

    mean_train_score-mean_test_score  
0                           0.008610  
1                           0.002246  
2                           0.000300  
3                           0.000223  
4                           0.001553  
5                           0.000551  
6                           0.000107  
7                           0.000007  
8                           0.000005  
9                           0.000005  
10                          0.000005  
11                          0.000005  
12                          0.000005  
13                          0.000005  
14                          0.000005  
15                          0.000005

In [8]:
# Train
plt.plot(results['param_alpha'], results['mean_train_score'], label="Train AUC")
plt.scatter(results['param_alpha'], results['mean_train_score'], label="Train AUC points")

# Validation
plt.plot(results['param_alpha'], results['mean_test_score'], label="Validation AUC")
plt.scatter(results['param_alpha'], results['mean_test_score'], label="Validation AUC points")

plt.xlabel("Alpha: hyperparameter")
plt.ylabel("AUC")
plt.title("AUC vs alpha curves")
plt.legend()
plt.show()

# 5. Prediction

In [9]:
alpha = 0.0001
final_model = model_class(alpha=alpha, **model_params)
final_model.fit(final_x_train, final_y_train)

final_y_train_pred = final_model.predict(final_x_train)
final_y_test_pred = final_model.predict(final_x_test)


# 7. Performance Measures

In [10]:
performance_measures = performance.get_performance_measures(final_model, final_x_test, final_y_test)
performance_measures

{'accuracy': 0.9330470904457709,
 'f1': 0.9189177763686325,
 'confusion_matrix':              actual_0  actual_1
 predicted_0     31721      1183
 predicted_1      2653     21737}

# Test

In [ ]:
predict.manual_test(final_model, 
                         model_id="SVM",
                         vectorizer=vectorizer)

In [12]:
test_results = predict.test_tweets_from_file(final_model,
                             "bot/tweets.json",
                             model_id="SVM",
                             vectorizer=vectorizer)

In [13]:
print(test_results['is_depressed(model output)'].value_counts())

0    133
1     73
Name: is_depressed(model output), dtype: int64


In [14]:
depressed_tweets = test_results[test_results['is_depressed(model output)'] == 1]

In [15]:
depressed_tweets.sort_values("model output probability (if any)", ascending=False).head(20)

Text  \
91     This man has been an idiot for his entire life.   
22   I wish it does because as of now i have hit th...   
178  Life is simple but we choose to complicate it ...   
21   LADbible blaming popcorn for nearly killing so...   
19   @starfish_72 I have no doubt that you will hel...   
174                                     What is life 😪   
18   It's some Derren Brown mind shit. I physically...   
14   Doctors don’t make you healthy. Teachers don’t...   
172               Life is rough so you gotta be tough.   
20   @acaucasianmom Ugh don't even get me started w...   
176  @Thia_Sparks if you want to lose your weight s...   
175  I want mykori u want milo ice kepal. We are no...   
184  I fucking love my icon it fully represents how...   
183  https://t.co/STbyLOYR5U if someone makes an ef...   
81   @ptrjytpz if you want to lose your weight so f...   
82          stan twt/real life https://t.co/MH3VyXiObh   
201  A life time ago it used to bother me what peop...   
200  Waaaait Naaaa Shorty You Selling A Vid Of You ...   
198  Can’t wait to bawl my eyes out when shonda KIL...   
75   You can’t even know what’s happening in that p...   

     is_depressed(model output) model output probability (if any)  
91                            1                              None  
22                            1                              None  
178                           1                              None  
21                            1                              None  
19                            1                              None  
174                           1                              None  
18                            1                              None  
14                            1                              None  
172                           1                              None  
20                            1                              None  
176                           1                              None  
175                           1                              None  
184                           1                              None  
183                           1                              None  
81                            1                              None  
82                            1                              None  
201                           1                              None  
200                           1                              None  
198                           1                              None  
75                            1                              None

# Future Work
- Explore string kernel https://github.com/timshenkao/StringKernelSVM
- word2vec using SVM (https://shop.tarjomeplus.com/UploadFileEn/TPLUS_EN_3959.pdf)
- https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/